In [161]:
import access_multiple_drives as acc
import pandas as pd
%load_ext autoreload
%autoreload 2
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [151]:
account1_creds = acc.auth(tokenFile = './token2.json')    
#mimeType Input: compressed_files, images,pdfs, videos
df=acc.list_drive_account_files(account1_creds,"videos")

In [152]:
df["size"] = pd.to_numeric(df["size"])
df=df[df.md5Checksum.isin(df.groupby("md5Checksum").filter(lambda x: len(x) > 1)["md5Checksum"])].sort_values(by=['size','md5Checksum','createdTime'],ascending=[False,True,True])

In [153]:
df

id                       name  \
267  1BXpKoHvsi-JWSxhS-F-64EgR2z9AwFxzrw  VID_20180327_193501_1.mp4   
266    13x-Vs1BpoFNR4WgzzT9a-wahBRrNdSUj  VID_20180327_193501_1.mp4   
64   1ShzGWg37QSyHWCzls7SgyUqkorVHmoUfCQ        20190704_113210.mp4   
58     1p9idmYpP0DGzJac_Ojz-lU6lx5qOrMTw        20190704_113210.mp4   
317  1CTKNqUMGIsmy5OwF17iK1YyO56qrNgUwRA    VID_20180303_124715.mp4   
..                                   ...                        ...   
32     1cBC2PfjyEj5KGXAIpxHMU8Q2Nks1gtLu        20190702_161520.mp4   
82   1UPe0hAshRjgfnclQJGiUS6PADkZ9nMaWuw        20190703_114719.mp4   
46     1AHYJ9wqAziigxrKiMu0Phz7avZ_iU13U        20190703_114719.mp4   
74   1YhmIaJhdhLHw6yLZCj_Eo-ern64FCSNdUw        20190703_114735.mp4   
44     1KQ1WAZY4YwgbdnLC9t7nKYK5vnesanKW        20190703_114735.mp4   

      mimeType               createdTime              modifiedTime  \
267  video/mp4  2018-03-27T17:35:01.000Z  2018-03-28T14:54:30.000Z   
266  video/mp4  2018-05-14T17:56:28.714Z  2018-03-28T14:54:30.000Z   
64   video/mp4  2019-07-04T09:46:17.000Z  2019-07-04T11:01:46.000Z   
58   video/mp4  2019-08-08T21:33:35.466Z  2019-08-08T19:45:54.000Z   
317  video/mp4  2018-03-03T10:47:15.000Z  2018-03-03T12:05:37.000Z   
..         ...                       ...                       ...   
32   video/mp4  2019-08-12T15:57:55.538Z  2019-08-08T19:46:46.000Z   
82   video/mp4  2019-07-03T09:47:22.000Z  2019-07-03T10:01:43.000Z   
46   video/mp4  2019-08-12T15:24:07.571Z  2019-08-08T19:46:26.000Z   
74   video/mp4  2019-07-03T09:47:37.000Z  2019-07-03T10:43:19.000Z   
44   video/mp4  2019-08-12T15:24:36.326Z  2019-08-08T19:46:27.000Z   

                          md5Checksum       size       account  
267  d652b6bb95dcef5f4d9ca51330916f95  627665271  main_account  
266  d652b6bb95dcef5f4d9ca51330916f95  627665271  main_account  
64   910b697c541242b3299da29f43a091e3  428330578  main_account  
58   910b697c541242b3299da29f43a091e3  428330578  main_account  
317  62f0d2dadf409a6df3a7029aaf69ccda  284757861  main_account  
..                                ...        ...           ...  
32   f52e1ecb64218d8c3aa62f20e9f8fa84     259447  main_account  
82   5498cf69cf7e20359e6afe4408ab62c3     217330  main_account  
46   5498cf69cf7e20359e6afe4408ab62c3     217330  main_account  
74   2a253c19f1f89d993e7689a914a20b5a     187543  main_account  
44   2a253c19f1f89d993e7689a914a20b5a     187543  main_account  

[113 rows x 8 columns]

In [70]:
## find the duplicated files(based on md5checksum) and return the rown indices without the first occurance 
duplicated_indices=df.duplicated(subset=['md5Checksum'],keep='first')
duplicated_indices=duplicated_indices[duplicated_indices != False].index.values.tolist()
duplicated_indices

[42,
 146,
 50,
 203,
 280,
 31,
 299,
 36,
 56,
 26,
 292,
 44,
 157,
 54,
 41,
 25,
 45,
 29,
 197,
 46,
 60,
 285,
 300,
 268,
 30,
 39,
 12,
 24,
 40,
 38,
 48,
 294,
 289,
 290,
 58,
 28,
 244,
 59,
 35,
 51,
 49,
 47,
 33,
 22,
 266,
 37,
 156,
 52,
 34,
 53,
 43,
 23,
 57,
 55,
 32,
 27,
 283]

In [80]:
## return the file ids for the duplicated files 
#df.loc[duplicated_indices,'id']

In [168]:
## access two accounts to get the files list by mimetype ( images, videos, pdfs, compressed)
def get_multiple_account_files_by_type(account1_creds,account2_creds,file_type="videos"):
    account1_creds = acc.auth(tokenFile = './token1.json')    
    account1_files_df=acc.list_drive_account_files(account1_creds,file_type)
    #account1_files_df
    if(account2_creds is not None):
        account2_creds = acc.auth(tokenFile = './token2.json')
        account2_files_df=acc.list_drive_account_files(account2_creds,file_type,"account_2")
        #account2_files_df

    ## merge the two accounts files into one data frame 
    
    return account2_files_df.append(account1_files_df,ignore_index=True)

In [171]:
get_multiple_account_files_by_type(account1_creds,account2_creds,file_type="images")

id  \
0      1F41SaICZucu3ZcC-GcRnAMge6xUWWJYy   
1      1ZKNRttxqXJXTF7A8q1aES6XufxIlQCCJ   
2      1sS9b_u4JXSYWm4gJAdT1b5lDE7Jp0pDv   
3      1WDY1VABeqWTTz5O14FGrdBKUCRdRKqzE   
4      1ZyLRuO5lC2w3sRvJe0G7fpT8Gx7WKhJR   
...                                  ...   
11763  1ln-naUv-P4YyIhIFAZPcgJ9jTK0dAwKL   
11764  1LkIdlb4xWXNtn4LIhq50njws5cZMx67E   
11765  1LQAfnEDKCsRUrrsHETOY9N3o9XahgRYs   
11766  1zEgwuZ6dip_KSrrzW5cGd7QrqXGpqTO1   
11767  1Nvd1xSx3wFj54bb5oWcI-fXiKHCN1P3v   

                                                   name    mimeType  \
0                                          IMG_1315.JPG  image/jpeg   
1                               IMG-20211229-WA0007.jpg  image/jpeg   
2                                          IMG_1150.JPG  image/jpeg   
3                                          IMG_1149.JPG  image/jpeg   
4              Finanzamt_letter_2_Nov_2021_IMG_1148.JPG  image/jpeg   
...                                                 ...         ...   
11763      WhatsApp Image 2019-02-01 at 7.31.13 PM.jpeg  image/jpeg   
11764  WhatsApp Image 2019-02-01 at 7.31.12 PM (1).jpeg  image/jpeg   
11765      WhatsApp Image 2019-02-01 at 7.31.12 PM.jpeg  image/jpeg   
11766      WhatsApp Image 2019-02-01 at 7.31.11 PM.jpeg  image/jpeg   
11767                            tropical Island UX.png   image/png   

                    createdTime              modifiedTime  \
0      2022-01-07T09:25:55.013Z  2022-01-07T09:25:55.013Z   
1      2021-12-29T17:42:57.389Z  2021-12-29T17:42:54.881Z   
2      2021-12-26T19:50:16.821Z  2021-12-26T19:50:16.821Z   
3      2021-12-26T19:49:56.259Z  2021-12-26T19:49:56.259Z   
4      2021-12-26T18:21:44.664Z  2021-12-26T18:22:11.381Z   
...                         ...                       ...   
11763  2019-02-01T18:38:40.727Z  2019-02-01T18:36:34.000Z   
11764  2019-02-01T18:38:41.109Z  2019-02-01T18:36:24.000Z   
11765  2019-02-01T18:38:47.032Z  2019-02-01T18:36:21.000Z   
11766  2019-02-01T18:38:48.393Z  2019-02-01T18:36:19.000Z   
11767  2019-01-28T18:19:52.424Z  2019-01-28T18:19:52.000Z   

                            md5Checksum     size       account  
0      f189674a9e1e50a0e4e4178ac2d0881c  3533200     account_2  
1      f78bd1e1dcc135ffbf87307a201129ce    73958     account_2  
2      a0d50f10ecad3f9f262518bcae9c80c0  3153470     account_2  
3      e66598fdde6a67075038e07c81ffdc3b  2839760     account_2  
4      0e464f92f119bd17dd731b44b1154266  3697712     account_2  
...                                 ...      ...           ...  
11763  8ea0263039dc815f05a1470dae76fb5d    56466  main_account  
11764  8df78391f836a00d5877561c1d450b29    52820  main_account  
11765  2b8f311452c5f102e3f8ef5f41462313    61550  main_account  
11766  1154cbbd67f55ae0ca41dc2a3e4d3e14    65569  main_account  
11767  03082c82e09a26c29e7d58736c2547c5  1043592  main_account  

[11768 rows x 8 columns]

In [173]:
## this function to return the Files Ids that needs to pass through too Drive API for deletiion.
## Input: files_list_df: teh data frame the contains all the files retrieved from Drive account. It includes all duplicates without grouping
## returns only the files Ids and teh accounts belongs to
def get_duplicated_files_ids(files_list_df):
    ## convert column size to integer for sorting purposes
    files_list_df["size"] = pd.to_numeric(files_list_df["size"])
    ## group the files by md5checksum and keep only the duplicated files. sorting them to keep the oldest created file as the first occurance version of the duplicaes
    files_list_df=files_list_df[files_list_df.md5Checksum.isin(files_list_df.groupby("md5Checksum").filter(lambda x: len(x) > 1)["md5Checksum"])].sort_values(by=['size','md5Checksum',"createdTime"],ascending=[False,True,True])
    
    ## find the duplicated files(based on md5checksum) and return the rown indices without the first occurance 
    duplicated_indices=files_list_df.duplicated(subset=['md5Checksum'],keep='first')
    duplicated_indices=duplicated_indices[duplicated_indices != False].index.values.tolist()
    #duplicated_indices
    
    ## return the file ids for the duplicated files that needs to be deleted
    return files_list_df.loc[duplicated_indices,['id',"account"]].values.tolist()

In [175]:
## Mimetypes:  ["application/pdf","image/jpeg","image/png","video/mp4","video/x-msvideo","video/x-ms-wmv","application/zip","application/rar","application/tar"]

files_list_df=get_multiple_account_files_by_type(account1_creds,account2_creds,file_type="videos")
get_duplicated_files_ids(files_list_df)

[['13x-Vs1BpoFNR4WgzzT9a-wahBRrNdSUj', 'account_2'],
 ['1p9idmYpP0DGzJac_Ojz-lU6lx5qOrMTw', 'account_2'],
 ['1Ldrd1hT32uqt81rXv2xfTD1zwwevV0Kv', 'account_2'],
 ['1u365hkJ45azXRlGxtmi3473EEeiKz34V', 'account_2'],
 ['1BMnuZgbOToaBdFRUtBFuLht6D9kH3w8K', 'account_2'],
 ['1-5_QUgz1AE6eIakyn-1GYLoL6YQW3-ZD', 'account_2'],
 ['1j5qBDKY-OLBQbf1ThjAPKkfcqpmk485r', 'account_2'],
 ['1mk-xJjgI7U0Pd8m0LLSIKS4t4WgREoRQ', 'account_2'],
 ['1JB42AIk2t9PhiQ0LuWrpb4Xr-RhZU6oN', 'account_2'],
 ['1eCf_1Av3MLrKYvh9IPWWAfzbbrguqR_c', 'account_2'],
 ['1r2g3hcchhWNrFMiDnJ8huoRhn0XeF4iT', 'account_2'],
 ['1V9f0-oHtFSa6pg8MKKFoviNc5lSX1Svj', 'account_2'],
 ['14wb20szpVlKHLC-SMUe1822UucUHrHs0', 'account_2'],
 ['1BeDQ_CO8Ha5QFH6yVNLH27TP_XXhsJUK', 'account_2'],
 ['17u-__11TQQ1NRoJFMBQhmfjGuuKv2PVZ', 'account_2'],
 ['1bMi73ZrZrBj2WBII5rZSLJsri8OMrnye', 'account_2'],
 ['1r3OJ3FDcRAlG_5Dq993oC0wdiK9Ox2nZ', 'account_2'],
 ['1jkS4fPJEZkxdT6Bx_CNSz6QUQYZSbop4', 'account_2'],
 ['1QU96TXwCqQycemVpbhbjBcMkQM7NOIkq', 'accoun

In [ ]:
from apiclient import errors

def delete_file(service, file_id):
  """Permanently delete a file, skipping the trash.

  Args:
    service: Drive API service instance.
    file_id: ID of the file to delete.
  """
  try:
    service.files().delete(fileId=file_id).execute()
  except errors.HttpError, error:
    print 'An error occurred: %s' % error